**Surface processing**

***Create layer surfaces***
Uses mris-inflate. Based on the number of surfaces you specify. Each will be approximately equidistant. Vertex correspondence is maintained, i.e each layer will have vertices along the vector connecting a white matter surface vertex with the corresponding pial surface vertex.

***Convert to gifti***

***Remove RAS offset***
Converts all surfaces from FS RSA space to native space for coregistration with original MRI

***Remove deep vertices***
Freesurfer operates on hemispheres independently, resulting in deep vertices and mesh faces cutting through subcortical structures. These are removed because they are not part of the cortical source space. Any vertex not labelled or labelled as unknown in the fs atlas are removed. This is applied to each hemisphere of each layer mesh.

***Combine hemispheres***
The left and right hemisphere meshes are combined by concatenation of their vertices and faces (left then right). No new faces are created. This is done for each layer.

***Downsample***
The surfaces are much too dense to use in source reconstruction. They must be downsampled, but we must maintain vertex correspondence between them. The pial surface is therefore downsampled by a factor of 10 using the vtkDecimatePro algorithm, which only removes vertices rather than creating new ones. The removed vertices are also removed from each other layer mesh, and the face structure from the pial mesh is copied to them (though the faces are not used in the source reconstruction if link vector orientations are used).

***Compute link vectors***
The dipole orientation at each source location is computed. Downsampled surface normals, original surface normals, cortical patch statistics, or link vectors can be specified. The first three compute vertex normals based on the orientation of the surrounding faces. They can be either computed separately for each layer, or the pial surface orientations can be used for all layers. The link vectors method computes vectors connecting corresponding pial and white matter vertices (pointing toward the white matter), which are therefore the same across layer. The orientation vectors are stored in the normals attribute of the gifti object.

***Combine layers***
The layer meshes are then combined into a single mesh by concatenating their vertices and faces (from pial to white matter). No new faces are created (i.e. there are no edges connecting vertices across layers)

In [7]:
import os
import sys
import subprocess
import numpy as np
import nibabel as nib
from surf import *

In [2]:
postprocess_freesurfer_surfaces(
    'sub-104',                                
    './test_output',                                
    'multilayer.2.ds.link_vector.gii',                                
    n_surfaces=2,                                 
    ds_factor=0.1,                                 
    orientation='link_vector',                                 
    remove_deep=True
)

[ 2.02536 36.9153  16.8828 ]
100.0% of the vertices in the decimated surface belong to the original surface.


In [3]:
postprocess_freesurfer_surfaces(
    'sub-104',                                
    './test_output',
    'multilayer.11.ds.link_vector.gii',
    n_surfaces=11, 
    ds_factor=0.1, 
    orientation='link_vector', 
    remove_deep=True
)

[ 2.02536 36.9153  16.8828 ]
100.0% of the vertices in the decimated surface belong to the original surface.


In [4]:
postprocess_freesurfer_surfaces(
    'sub-104',                                
    './test_output',
    'multilayer.32.ds.link_vector.gii',
    n_surfaces=32, 
    ds_factor=0.1, 
    orientation='link_vector', 
    remove_deep=True
)

[ 2.02536 36.9153  16.8828 ]
100.0% of the vertices in the decimated surface belong to the original surface.
